In [ ]:
import torch
import numpy as np
from pathlib import Path
from torch.utils.data import Dataset
import numpy as np
import pandas as pd
label_map = {
    'affected_individuals': 0,
    'infrastructure_and_utility_damage': 1,
    'not_humanitarian': 2,
    'other_relevant_information': 3,
    'rescue_volunteering_or_donation_effort': 4,
    'vehicle_damage': 1,
    'injured_or_dead_people': 0,
    'missing_or_found_people': 0
}

In [68]:


class NPZLoader(Dataset):
    def __init__(self, img_embed_filename,text_emb_filename,tsv_file, transform=None):
#        self.files = list(Path(path).glob('*/*.npz'))
#        self.transform = transform
        self.text_embed = torch.load(text_embed_filename).mean(axis=1) # ../sporwal9/eng_test_data_emb.pt
        self.img_emb = np.load(img_embed_filename) #image_model_ouputs/resnet18_image_embedding_512_test.npz
        self.original_df = pd.read_csv(tsv_file, sep='\t') #'../sverma324/data/crisis-mmd/task_humanitarian_text_img_train.tsv'
        self.df = self.original_df.loc[original_df.label_text_image=="Positive"]
        id_list=  list(self.df['tweet_id'].unique())
    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        image_tweetID = self.img_emb['image_tweetID'][idx]
        
        image_embedding = torch.from_numpy(self.img_emb['image_embedding'][idx].squeeze(1))
        
        text_embedding = self.text_embed[idx].unsqueeze(1)
        fusion_emb = torch.cat((text_embedding.cuda(),image_embedding.cuda()),dim=0)
        label = self.df.loc[self.df['tweet_id']==image_tweetID,'label'][0]
        if image_tweetID in id_list :
            return fusion_emb,image_tweetID,label


In [89]:
df.loc[df['tweet_id']==905952332923338752,'label'].values

array(['rescue_volunteering_or_donation_effort'], dtype=object)

In [80]:
tsv_file = '../sverma324/data/crisis-mmd/task_humanitarian_text_img_test.tsv'
original_df = pd.read_csv(tsv_file, sep='\t') 
df = original_df.loc[original_df.label_text_image=="Positive"]
# df[df['tweet_id']=='905952332923338752','label']
df

,event_name,tweet_id,image_id,tweet_text,image,label,label_text,label_image,label_text_image
0,hurricane_harvey,905952332923338752,905952332923338752_0,.@Lendio has a great event tomorrow for both #...,data_image/hurricane_harvey/8_9_2017/905952332...,rescue_volunteering_or_donation_effort,rescue_volunteering_or_donation_effort,rescue_volunteering_or_donation_effort,Positive
1,mexico_earthquake,912022130396672000,912022130396672000_0,Now @ReposadoBar also Carlo surf band at 8pm #...,data_image/mexico_earthquake/24_9_2017/9120221...,not_humanitarian,not_humanitarian,not_humanitarian,Positive
3,hurricane_maria,927758858776465408,927758858776465408_0,Guaynabo resident Efrain Diaz stands by a brid...,data_image/hurricane_maria/7_11_2017/927758858...,infrastructure_and_utility_damage,infrastructure_and_utility_damage,infrastructure_and_utility_damage,Positive
4,hurricane_harvey,907654309587779584,907654309587779584_0,"After Harvey, Irma Leaves Its Own Trail of Des...",data_image/hurricane_harvey/12_9_2017/90765430...,other_relevant_information,other_relevant_information,other_relevant_information,Positive
5,hurricane_maria,913745586104528898,913745586104528898_0,Dawn just won $400 and we're gonna match the a...,data_image/hurricane_maria/29_9_2017/913745586...,not_humanitarian,not_humanitarian,not_humanitarian,Positive
...,...,...,...,...,...,...,...,...,...
2228,iraq_iran_earthquake,930415579294584834,930415579294584834_0,Iran: Survivors struggle in aftermath of year'...,data_image/iraq_iran_earthquake/14_11_2017/930...,affected_individuals,affected_individuals,affected_individuals,Positive
2229,california_wildfires,923088359089688577,923088359089688577_0,#LosAngelesTimes | The package includes $576.5...,data_image/california_wildfires/25_10_2017/923...,not_humanitarian,not_humanitarian,not_humanitarian,Positive
2231,mexico_earthquake,910670262789709824,910670262789709824_0,S A F E B O Y E S ready to look for people aft...,data_image/mexico_earthquake/21_9_2017/9106702...,rescue_volunteering_or_donation_effort,rescue_volunteering_or_donation_effort,rescue_volunteering_or_donation_effort,Positive
2232,california_wildfires,921130711863631872,921130711863631872_0,#thesciencemic NASA Map of Wildfire Damage Hel...,data_image/california_wildfires/19_10_2017/921...,other_relevant_information,other_relevant_information,other_relevant_information,Positive


In [3]:
import torch

In [4]:
txt_emb = torch.load('../sporwal9/eng_test_data_emb.pt')
# txt_emb.shape

In [58]:
txt = txt_emb.mean(axis=1)[0].unsqueeze(1)

In [74]:
img_emb = np.load("image_model_ouputs/resnet18_image_embedding_512_test.npz")


In [75]:
image_embedding, image_tweetID = img_emb['image_embedding'],img_emb['image_tweetID']

In [63]:
img = torch.from_numpy(image_embedding[0].squeeze(1))

In [64]:
txt.shape,img.shape

(torch.Size([768, 1]), torch.Size([512, 1]))

In [67]:
torch.cat((txt.cuda(),img.cuda()),dim=0).shape

torch.Size([1280, 1])

In [11]:
len(a['image_embedding'])

40824

In [43]:
original_df.shape

(6126, 9)

In [77]:
len(image_tweetID)

2237

In [ ]:
train_dataset = NPZLoader()